In [1]:
import read_20newsgroup_stemmed as ng
import tensorflow as tf
from gensim.models import KeyedVectors
import numpy as np

In [2]:
#to read the data
data = ng.ReadNewsGroup('./data/', rare_threshold=5, seed=10)

In [ ]:
##pretrained word2vec model
word_vec = KeyedVectors.load_word2vec_format('./data/wiki.en.vec')

In [ ]:
#parameters
batch_size = 50
max_length = 430
word_vec_size = 300
num_batches = 1000
num_classes = 20

In [27]:
# Define the function from words to vectors
def word2vec(sentences):
    n = len(sentences)
    word_vecs = np.zeros((batch_size, max_length, word_vec_size))
    skipped = 0
    total = 0
    for i in range(n):
        s = sentences[i]
        m = len(s)
        total += m
        k = 0
        for j in range(max_length - m, max_length):
            try:
                word_vecs[i, j, :] = word_vec[s[k]]
            except:
                skipped += 1
            k += 1
    #to count miss word in vocabulary
    print('Skipped', skipped, 'out of', total)
    return word_vecs

In [28]:
try:
    del lstm_size
    del lstm
    del hidden_state
    del current_state
    del words
    del state
    del output
    tf.reset_default_graph()
except:
    pass
    
lstm_size = 200
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
hidden_state = tf.zeros([batch_size, lstm_size])
current_state = tf.zeros([batch_size, lstm_size])
state = hidden_state, current_state

words = tf.placeholder(tf.float32, [batch_size, max_length, word_vec_size])
labels = tf.placeholder(tf.float32, [batch_size, num_classes])

output = tf.zeros([batch_size, lstm_size])
with tf.variable_scope("RNN"):
    for i in range(max_length):
        if i > 0: tf.get_variable_scope().reuse_variables()
        output, state = lstm(tf.reshape(words[:, i, :], \
                                        [batch_size, -1]), state)

output = tf.layers.dense(output, num_classes)
pred = tf.nn.softmax(output)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                    logits=output, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
train_op = optimizer.minimize(loss_op)

In [29]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [30]:
for i in range(num_batches):
    e, l = data.get_data(batch_size, set_type='train')
    batch_x = []
    batch_y = []
    for i in range(batch_size):
        email = e[i]
        label = l[i]
        if len(email) <= max_length:
            batch_x.append(email)
            batch_y.append(label)
            
    batch_x = word2vec(batch_x)
    batch_y = np.asarray(batch_y)
    temp = np.zeros((batch_size, num_classes))
    n = batch_y.shape[0]
    for i in range(n):
        temp[i, batch_y[i]] = 1
    for i in range(n, batch_size):
        temp[i, 0] = 1
    batch_y = temp
    
    _, cost = sess.run([train_op, loss_op], feed_dict={words: batch_x,\
                                                    labels: batch_y})
    
    print(cost)
    

Skipped 182 out of 3005
2.97571
Skipped 153 out of 2926
3.01638
Skipped 183 out of 2833
3.01051
Skipped 210 out of 3064
2.99382
Skipped 318 out of 3397
2.91766
Skipped 170 out of 2896
2.93258
Skipped 223 out of 3387
2.97402
Skipped 204 out of 3821
2.82472
Skipped 144 out of 2523
2.97719
Skipped 174 out of 3673
2.82332
Skipped 141 out of 2585
2.91842
Skipped 185 out of 3034
2.82127
Skipped 172 out of 3242
2.77361
Skipped 211 out of 3125
2.74627
Skipped 208 out of 3397
2.58033
Skipped 203 out of 2777
2.50943
Skipped 225 out of 3404
2.85472
Skipped 213 out of 3555
2.5967
Skipped 190 out of 3679
2.61398
Skipped 145 out of 2496
2.58282
Skipped 256 out of 3921
2.55522
Skipped 187 out of 3444
2.66536
Skipped 134 out of 2557
2.54335
Skipped 151 out of 3002
2.38878
Skipped 156 out of 2966
2.45196
Skipped 129 out of 2815
2.47395
Skipped 136 out of 3534
2.43606
Skipped 209 out of 4218
2.29965
Skipped 124 out of 2557
2.41637
Skipped 155 out of 2956
2.31419
Skipped 205 out of 3555
2.33911
Skipped 2

1.75173
Skipped 178 out of 3133
1.56445
Skipped 161 out of 2958
1.43889
Skipped 304 out of 4213
1.73252
Skipped 246 out of 3306
1.57191
Skipped 104 out of 2278
1.82687
Skipped 143 out of 2799
1.33933
Skipped 273 out of 3957
1.61453
Skipped 135 out of 2510
1.73205
Skipped 229 out of 3073
1.27246
Skipped 196 out of 4273
1.39584
Skipped 204 out of 2936
1.62922
Skipped 140 out of 2617
1.72511
Skipped 166 out of 3425
1.7032
Skipped 216 out of 3201
1.5836
Skipped 263 out of 3872
1.60289
Skipped 133 out of 2678
1.70886
Skipped 148 out of 2686
1.68497
Skipped 109 out of 1972
1.93976
Skipped 230 out of 3543
1.74396
Skipped 133 out of 2428
1.47922
Skipped 194 out of 3559
1.35245
Skipped 191 out of 3621
1.50656
Skipped 239 out of 4366
1.53538
Skipped 130 out of 2869
1.4228
Skipped 219 out of 3663
1.31306
Skipped 210 out of 3338
1.57909
Skipped 213 out of 3497
1.6763
Skipped 172 out of 3605
1.82517
Skipped 148 out of 2971
1.59863
Skipped 214 out of 3468
1.53867
Skipped 152 out of 2421
1.80586
Skip

1.669
Skipped 216 out of 3634
1.52992
Skipped 201 out of 3445
1.41136
Skipped 191 out of 4315
1.13723
Skipped 145 out of 2717
1.10065
Skipped 169 out of 3273
1.32261
Skipped 195 out of 4692
1.46516
Skipped 208 out of 3263
1.14113
Skipped 249 out of 3893
1.30005
Skipped 176 out of 3961
1.25673
Skipped 132 out of 2497
1.24457
Skipped 223 out of 3690
1.35818
Skipped 162 out of 2936
1.15441
Skipped 173 out of 3381
1.45944
Skipped 149 out of 2582
1.6253
Skipped 129 out of 2650
1.32084
Skipped 266 out of 3370
1.08587
Skipped 180 out of 3104
1.46189
Skipped 168 out of 2570
1.12607
Skipped 170 out of 2560
1.12709
Skipped 183 out of 3755
1.2098
Skipped 108 out of 2389
1.22798
Skipped 198 out of 2989
1.57321
Skipped 196 out of 3326
1.55663
Skipped 121 out of 2246
1.54225
Skipped 229 out of 3379
1.87535
Skipped 205 out of 3911
1.39921
Skipped 183 out of 3408
1.54913
Skipped 189 out of 3470
1.16221
Skipped 169 out of 2989
1.27034
Skipped 159 out of 2590
1.32687
Skipped 154 out of 3028
1.50851
Skip

1.47211
Skipped 277 out of 3064
1.62089
Skipped 203 out of 3522
1.47418
Skipped 216 out of 3511
1.61179
Skipped 180 out of 3812
1.55307
Skipped 157 out of 3060
1.31593
Skipped 180 out of 3097
1.62604
Skipped 220 out of 3983
1.70464
Skipped 214 out of 3638
1.44864
Skipped 201 out of 3617
1.47968
Skipped 130 out of 2653
1.50463
Skipped 133 out of 3367
1.38745
Skipped 248 out of 3787
1.44284
Skipped 275 out of 4785
1.23688
Skipped 209 out of 3556
1.59423
Skipped 138 out of 2507
1.67217
Skipped 145 out of 2139
1.24122
Skipped 132 out of 3077
1.58327
Skipped 183 out of 3186
1.38692
Skipped 248 out of 4025
1.40731
Skipped 173 out of 3090
1.4848
Skipped 259 out of 4009
1.28074
Skipped 139 out of 3009
1.20102
Skipped 167 out of 2764
1.30569
Skipped 207 out of 3796
1.00506
Skipped 159 out of 3061
1.10468
Skipped 197 out of 3424
1.0978
Skipped 197 out of 3964
1.46175
Skipped 118 out of 2575
1.30738
Skipped 97 out of 2440
1.50669
Skipped 179 out of 3843
1.22135
Skipped 120 out of 2694
1.44697
Ski

In [23]:
for i in range(num_batches):
    sum=0
    e, l = data.get_data(batch_size, set_type='test')
    batch_x = []
    batch_y = []
    for i in range(batch_size):
        email = e[i]
        label = l[i]
        if len(email) <= max_length:
            batch_x.append(email)
            batch_y.append(label)
            
    batch_x = word2vec(batch_x)
    batch_y = np.asarray(batch_y)
    temp = np.zeros((batch_size, num_classes))
    n = batch_y.shape[0]
    for i in range(n):
        temp[i, batch_y[i]] = 1
    for i in range(n, batch_size):
        temp[i, 0] = 1
    batch_y = temp
    p= sess.run(pred, feed_dict={words: batch_x})
    idx=np.argmax(p,axis=1)
    true_idx=np.argmax(batch_y,axis=1)
    acc=np.mean(idx==true_idx)
    sum=sum+acc

Skipped 205 out of 3952
Skipped 160 out of 3100
Skipped 157 out of 2989
Skipped 125 out of 2805
Skipped 161 out of 3192
Skipped 154 out of 2914
Skipped 150 out of 2552
Skipped 186 out of 3340
Skipped 187 out of 3029
Skipped 137 out of 3026
Skipped 209 out of 3365
Skipped 263 out of 4280
Skipped 198 out of 2817
Skipped 225 out of 3585
Skipped 178 out of 2824
Skipped 198 out of 4054
Skipped 137 out of 2675
Skipped 135 out of 2871
Skipped 207 out of 2965
Skipped 133 out of 2566
Skipped 212 out of 3600
Skipped 178 out of 3399
Skipped 157 out of 2867
Skipped 176 out of 3121
Skipped 217 out of 3828
Skipped 133 out of 2777
Skipped 147 out of 3315
Skipped 225 out of 3773
Skipped 152 out of 2771
Skipped 193 out of 3379
Skipped 201 out of 2852
Skipped 125 out of 3294
Skipped 156 out of 3281
Skipped 112 out of 1951
Skipped 240 out of 3845
Skipped 200 out of 3824
Skipped 192 out of 2688
Skipped 188 out of 3160
Skipped 176 out of 2615
Skipped 168 out of 4037
Skipped 205 out of 3271
Skipped 144 out 

Skipped 248 out of 3603
Skipped 131 out of 2371
Skipped 134 out of 2511
Skipped 166 out of 3528
Skipped 165 out of 3000
Skipped 170 out of 2854
Skipped 152 out of 3068
Skipped 147 out of 3105
Skipped 149 out of 2135
Skipped 142 out of 3331
Skipped 170 out of 2962
Skipped 160 out of 2522
Skipped 150 out of 2930
Skipped 175 out of 3732
Skipped 268 out of 3870
Skipped 280 out of 4651
Skipped 116 out of 2283
Skipped 138 out of 3073
Skipped 155 out of 2888
Skipped 285 out of 3600
Skipped 148 out of 2595
Skipped 202 out of 4172
Skipped 134 out of 2883
Skipped 191 out of 3376
Skipped 207 out of 3486
Skipped 124 out of 2718
Skipped 168 out of 3332
Skipped 149 out of 2978
Skipped 155 out of 3421
Skipped 256 out of 3251
Skipped 153 out of 2482
Skipped 171 out of 3029
Skipped 158 out of 3170
Skipped 158 out of 2331
Skipped 158 out of 3154
Skipped 120 out of 2581
Skipped 147 out of 2821
Skipped 192 out of 3147
Skipped 190 out of 3066
Skipped 247 out of 4336
Skipped 151 out of 2941
Skipped 147 out 

Skipped 119 out of 2697
Skipped 163 out of 2270
Skipped 125 out of 3036
Skipped 165 out of 2964
Skipped 164 out of 2616
Skipped 166 out of 3121
Skipped 201 out of 3629
Skipped 140 out of 2665
Skipped 194 out of 3112
Skipped 148 out of 2782
Skipped 158 out of 3182
Skipped 119 out of 2177
Skipped 178 out of 2680
Skipped 103 out of 2711
Skipped 155 out of 2723
Skipped 122 out of 3014
Skipped 225 out of 3400
Skipped 192 out of 3802
Skipped 128 out of 2749
Skipped 145 out of 2995
Skipped 275 out of 3094
Skipped 163 out of 3762
Skipped 241 out of 3351
Skipped 112 out of 2195
Skipped 171 out of 2851
Skipped 180 out of 2825
Skipped 115 out of 2280
Skipped 187 out of 3821
Skipped 146 out of 3256
Skipped 190 out of 3224
Skipped 184 out of 2731
Skipped 245 out of 3183
Skipped 174 out of 2830
Skipped 171 out of 3310
Skipped 185 out of 3423
Skipped 179 out of 2759
Skipped 197 out of 3477
Skipped 195 out of 3312
Skipped 154 out of 2744
Skipped 157 out of 2744
Skipped 220 out of 3406
Skipped 134 out 

In [25]:
total_acc=sum/1000
print(total_acc)

0.00068


In [22]:

#for checking accuracy on training data
""""
for i in range(num_batches):
    e, l = data.get_data(batch_size, set_type='train')
    batch_x = []
    batch_y = []
    for i in range(batch_size):
        email = e[i]
        label = l[i]
        if len(email) <= max_length:
            batch_x.append(email)
            batch_y.append(label)
            
    batch_x = word2vec(batch_x)
    batch_y = np.asarray(batch_y)
    temp = np.zeros((batch_size, num_classes))
    n = batch_y.shape[0]
    for i in range(n):
        temp[i, batch_y[i]] = 1
    for i in range(n, batch_size):
        temp[i, 0] = 1
    batch_y = temp
    p= sess.run(pred, feed_dict={words: batch_x})
    idx=np.argmax(p,axis=1)
    true_idx=np.argmax(batch_y,axis=1)
    print(np.mean(idx==true_idx))
"""

Skipped 180 out of 2787
0.68
Skipped 193 out of 3202
0.74
Skipped 190 out of 3342
0.66
Skipped 146 out of 2941
0.54
Skipped 184 out of 2944
0.7
Skipped 179 out of 3172
0.7
Skipped 150 out of 3016
0.76
Skipped 175 out of 2622
0.58
Skipped 182 out of 2551
0.66
Skipped 153 out of 2729
0.78
Skipped 124 out of 2845
0.76
Skipped 194 out of 2891
0.72
Skipped 173 out of 3222
0.6
Skipped 215 out of 3430
0.66
Skipped 140 out of 2391
0.66
Skipped 347 out of 4147
0.66
Skipped 192 out of 3458
0.62
Skipped 182 out of 3077
0.7
Skipped 147 out of 2689
0.64
Skipped 149 out of 2455
0.64
Skipped 163 out of 2870
0.68
Skipped 201 out of 3544
0.72
Skipped 201 out of 3798
0.62
Skipped 187 out of 2650
0.68
Skipped 153 out of 3011
0.56
Skipped 143 out of 3039
0.74
Skipped 201 out of 2621
0.7
Skipped 177 out of 3099
0.68
Skipped 185 out of 3158
0.66
Skipped 187 out of 3685
0.72
Skipped 156 out of 2948
0.62
Skipped 190 out of 3390
0.66
Skipped 193 out of 3847
0.84
Skipped 195 out of 2761
0.74
Skipped 127 out of 

0.68
Skipped 220 out of 3653
0.56
Skipped 159 out of 2877
0.64
Skipped 121 out of 2707
0.78
Skipped 138 out of 2052
0.66
Skipped 157 out of 3372
0.66
Skipped 209 out of 3450
0.56
Skipped 183 out of 3232
0.58
Skipped 146 out of 3114
0.72
Skipped 193 out of 3410
0.68
Skipped 140 out of 2977
0.64
Skipped 179 out of 3512
0.74
Skipped 178 out of 2728
0.74
Skipped 164 out of 3491
0.74
Skipped 164 out of 2382
0.7
Skipped 131 out of 2606
0.62
Skipped 176 out of 3645
0.56
Skipped 194 out of 2811
0.78
Skipped 221 out of 3215
0.64
Skipped 183 out of 3343
0.68
Skipped 215 out of 3814
0.8
Skipped 194 out of 2905
0.7
Skipped 94 out of 1670
0.82
Skipped 136 out of 2896
0.64
Skipped 153 out of 3170
0.68
Skipped 235 out of 3331
0.58
Skipped 174 out of 3634
0.68
Skipped 215 out of 4380
0.64
Skipped 118 out of 2588
0.66
Skipped 207 out of 3165
0.76
Skipped 163 out of 3397
0.56
Skipped 126 out of 2323
0.64
Skipped 188 out of 3001
0.7
Skipped 134 out of 3142
0.78
Skipped 217 out of 4524
0.58
Skipped 145 ou

0.76
Skipped 183 out of 2936
0.68
Skipped 180 out of 3780
0.68
Skipped 158 out of 3503
0.7
Skipped 135 out of 2955
0.62
Skipped 233 out of 3572
0.62
Skipped 141 out of 2668
0.66
Skipped 122 out of 3037
0.72
Skipped 189 out of 3101
0.62
Skipped 164 out of 2774
0.76
Skipped 217 out of 3676
0.64
Skipped 211 out of 3559
0.76
Skipped 165 out of 3056
0.7
Skipped 186 out of 2676
0.56
Skipped 210 out of 3305
0.66
Skipped 174 out of 3354
0.64
Skipped 211 out of 3690
0.64
Skipped 157 out of 2935
0.62
Skipped 198 out of 3161
0.7
Skipped 146 out of 3536
0.62
Skipped 181 out of 3054
0.76
Skipped 220 out of 3768
0.68
Skipped 178 out of 3698
0.56
Skipped 199 out of 3056
0.64
Skipped 152 out of 3077
0.64
Skipped 212 out of 3627
0.74
Skipped 205 out of 3081
0.7
Skipped 170 out of 3073
0.68
Skipped 187 out of 2962
0.62
Skipped 150 out of 3079
0.62
Skipped 176 out of 3322
0.8
Skipped 137 out of 2787
0.74
Skipped 159 out of 2963
0.64
Skipped 165 out of 3350
0.74
Skipped 192 out of 3275
0.7
Skipped 142 out

0.66
Skipped 200 out of 3011
0.54
Skipped 143 out of 3332
0.72
Skipped 128 out of 2802
0.7
Skipped 140 out of 3409
0.66
Skipped 103 out of 2490
0.58
Skipped 159 out of 2888
0.6
Skipped 142 out of 2889
0.82
Skipped 190 out of 3094
0.52
Skipped 210 out of 3379
0.64
Skipped 148 out of 3049
0.68
Skipped 197 out of 3747
0.6
Skipped 106 out of 2797
0.72
Skipped 245 out of 2975
0.72
Skipped 194 out of 3121
0.74
Skipped 216 out of 4368
0.74
Skipped 149 out of 2959
0.66
Skipped 144 out of 3274
0.58
Skipped 193 out of 3828
0.7
Skipped 233 out of 4204
0.76
Skipped 179 out of 3097
0.62
Skipped 187 out of 3409
0.68
Skipped 191 out of 2873
0.64
Skipped 163 out of 3298
0.66
Skipped 104 out of 2151
0.82
Skipped 214 out of 3348
0.66
Skipped 186 out of 2808
0.68
Skipped 134 out of 3206
0.64
Skipped 204 out of 3130
0.62
Skipped 187 out of 3241
0.76
Skipped 209 out of 3853
0.72
Skipped 285 out of 4184
0.66
Skipped 175 out of 3413
0.68
Skipped 135 out of 2513
0.66
Skipped 218 out of 3831
0.56
Skipped 185 o